### 整体结构
- 全连接神经网络（Fully-connected Neural Network, FCN），相邻层的所有神经元之间都有连接，这称为**全连接（fully-connected）**。另外，我们用Affine层实现了全连接层。
<div align="center">   <img src="data_mk/mk-2025-11-06-20-52-59.png" width="50%" style="margin: 20px 0;" /> </div>

 > 全连接的神经网络中，Affine层后面跟着激活函数ReLU层（或者Sigmoid层）。这里堆叠了4层“Affine-ReLU”组合，然后第5层是Affine层，最后由Softmax层输出最终结果（概率）。
- 卷积神经网络（Convolutional Neural Network, CNN）中新增了Convolution层和Pooling层。CNN的层的连接顺序是“**Convolution - ReLU -（Pooling）** ”（Pooling层有时会被省略）。这可以理解为之前的“Affine - ReLU”连接被替换成了“Convolution - ReLU -（Pooling）”连接。
<div align="center">   <img src="data_mk/mk-2025-11-06-20-57-11.png" width="50%" style="margin: 20px 0;" /> </div>

> 还需要注意的是，靠近输出的层中使用了之前的“Affine - ReLU”组合。此外，最后的输出层中使用了之前的“Affine - Softmax”组合。
<div align="center">

| 位置     | 使用结构                  | 目的              |
| ------ | --------------------- | --------------- |
| 卷积层前端  | Conv - ReLU - Pooling | 提取局部特征          |
| 靠近输出的层 | Affine - ReLU         | 综合全局特征，形成高层语义表示 |
| 最终输出层  | Affine - Softmax      | 输出类别概率，完成分类     |

</div>

- 对比

<div align="center">

| 特性     | 全连接网络（FCN）      | 卷积网络（CNN）      |
| ------ | --------------- | -------------- |
| 参数连接方式 | 每个神经元与下一层所有节点相连 | 局部连接（只连接局部区域）  |
| 参数量    | 非常多             | 大幅减少（共享权重）     |
| 特征提取   | 无法自动提取空间特征(数据被拉平为一维)      | 能自动提取图像局部与空间特征 |
| 应用场景   | 一般数据、表格类任务      | 图像、视频、语音等高维数据  |

</div>

- CNN中，将卷积层的输入输出数据称为特征图（feature map）。其中，卷积层的输入数据称为输入特征图（input feature map），输出数据称为输出特征图（output feature map）

### 卷积层
- **卷积运算**：对于输入数据，卷积运算以一定间隔滑动滤波器的窗口并应用。将各个位置上滤波器的元素和输入的对应元素相乘，然后再求和。然后，将这个结果保存到输出的对应位置。将这个过程在所有位置都进行一遍，就可以得到卷积运算的输出。
> CNN中，滤波器的参数就对应之前的权重。并且，CNN中也存在偏置。
<div align="center">   <img src="data_mk/mk-2025-11-06-21-08-29.png" width="50%" style="margin: 20px 0;" /> </div>

- **填充**：在进行卷积层的处理之前，有时要向输入数据的周围填入固定的数据（比如0等），这称为填充（padding）。
<div align="center">   <img src="data_mk/mk-2025-11-06-21-12-57.png" width="50%" style="margin: 20px 0;" /> </div>

> 如图所示，通过填充，大小为(4,4)的输入数据变成了(6,6)的形状。然后，应用大小为(3,3)的滤波器，生成了大小为(4,4)的输出数据。这个例子中将填充设成了1，不过填充的值也可以设置成2、3等任意的整数。
> 使用填充主要是为了调整输出的大小。如果每次进行卷积运算都会缩小空间，那么在某个时刻输出大小就有可能变为1，导致无法再应用卷积运算。

- **步幅**：应用滤波器的位置间隔称为步幅（stride）。
<div align="center">   <img src="data_mk/mk-2025-11-06-21-15-55.png" width="40%" style="margin: 20px 0;" /> </div>

- **如何计算输出大小**：假设输入大小为$(H,W)$，滤波器大小为$(FH,FW)$，输出大小为$(OH,OW)$，填充为$P$，步幅为$S$。  
  计算公式：
$$
OH = \frac{H+2P-FH}{S}+1
$$
$$
OW = \frac{W+2P-FW}{S}+1
$$

- **三维卷积运算**：通道方向上有多个特征图时，会按通道进行输入数据和滤波器的卷积运算，并将结果相加，从而得到输出。
<div align="center">   <img src="data_mk\mk-2025-11-06-21-27-27.png" width="55%" style="margin: 20px 0;" /> </div>

> 输入数据和滤波器的通道数一致。
- 结合方块思考。
  <div align="center">   <img src="data_mk\mk-2025-11-06-21-29-48.png" width="55%" style="margin: 20px 0;" /> </div>

  如果要在通道方向上也拥有多个卷积运算的输出，该怎么做呢？为此，就需要用到多个滤波器（权重）。
  <div align="center">   <img src="data_mk\mk-2025-11-06-21-30-42.png" width="55%" style="margin: 20px 0;" /> </div>

> 通过应用FN个滤波器，输出特征图也生成了FN个。如果将这FN个特征图汇集在一起，就得到了形状为(FN,OH,OW)的方块。将这个方块传给下一层，就是CNN的处理流。  
> 滤波器的权重数据要按(output_channel,input_channel, height, width)的顺序书写。比如，通道数为$3$、大小为$5×5$的滤波器有$20$个时，可以写成$(20,3,5,5)$。
<div align="center">   <img src="data_mk\mk-2025-11-06-21-34-27.png" width="55%" style="margin: 20px 0;" /> </div>

> 每个通道只有一个偏置。这里，偏置的形状是$(FN,1,1)$，滤波器的输出结果的形状是$(FN,OH,OW)$。这两个方块相加时，要对滤波器的输出结果$(FN,OH,OW)$按通道加上相同的偏置值。另外，不同形状的方块相加时，可以基于NumPy的广播功能轻松实现。
- **批处理**：神经网络的处理中进行了将输入数据打包的批处理。为此，需要将在各层间传递的数据保存为4维数据。具体地讲，就是按(batch_num,channel,height,width)的顺序保存数据。  
  卷积运算的处理流（批处理）：
<div align="center">   <img src="data_mk\mk-2025-11-06-21-38-24.png" width="55%" style="margin: 20px 0;" /> </div>

### 池化层
- **池化**是缩小高、长方向上的空间的运算。如图所示，进行将$2×2$的区域集约成$1$个元素的处理，缩小空间大小（Max池化）。
<div align="center">   <img src="data_mk\mk-2025-11-06-21-40-25.png" width="55%" style="margin: 20px 0;" /> </div>

> 一般来说，池化的窗口大小会和步幅设定成相同的值。
#### **池化层的特征**
1. **没有要学习的参数**：池化层和卷积层不同，没有要学习的参数。池化只是从目标区域中取最大值（或者平均值），所以不存在要学习的参数。
2. **通道数不发生变化**：经过池化运算，输入数据和输出数据的通道数不会发生变化。
3. **对微小的位置变化具有鲁棒性（健壮）**：对微小的位置变化具有鲁棒性（健壮）。
#### **常见的池化方式**
<div align="center">

| 池化方式                               | 计算方法               | 特点             | 优点                 | 缺点         | 常见应用                          |
| ---------------------------------- | ------------------ | -------------- | ------------------ | ---------- | ----------------------------- |
| **Max Pooling（最大池化）**              | 从目标区域中取出**最大值**    | 保留显著特征（如边缘、纹理） | 计算简单，保留关键信息，对平移不敏感 | 可能丢失细节信息   | 图像识别、卷积神经网络主流用法               |
| **Average Pooling（平均池化）**          | 计算目标区域中所有值的**平均值** | 平滑特征图          | 降低噪声、保持平滑过渡        | 可能削弱显著特征   | 早期CNN（如LeNet）或特征平滑任务          |
| **Global Average Pooling（全局平均池化）** | 对整个特征图取平均值         | 将每个特征图压缩为一个数   | 无需全连接层，减少参数，防止过拟合  | 可能损失空间位置信息 | 现代轻量CNN（如GoogLeNet、ResNet）输出层 |
| **Global Max Pooling（全局最大池化）**     | 对整个特征图取最大值         | 取出最强响应特征       | 保留最显著的全局特征         | 容易忽略次要特征   | 注意力机制、显著性检测                   |
| **L2 Pooling（平方平均池化）**             | 对区域内平方求和再开方        | 平衡最大与平均        | 保留强特征同时抑制噪声        | 计算复杂度高     | 特殊场景（如部分视觉模型）                 |

</div>



### 基于im2col的展开
- 如果老老实实地实现卷积运算，估计要重复好几层的for语句。这样的实现有点麻烦，而且，NumPy中存在使用for语句后处理变慢的缺点（NumPy中，访问元素时最好不要用for语句）。这里，我们不使用for语句，而是使
用im2col这个便利的函数进行简单的实现。
<div align="center">

| 项目   | 传统卷积      | 使用 `im2col` 的卷积 |
| ---- | --------- | --------------- |
| 运算形式 | 嵌套循环滑动窗口  | 矩阵乘法            |
| 优化空间 | 访问不连续，难并行 | 可调用高性能 GEMM 库   |
| 内存开销 | 小         | 较大（数据展开）        |
| 适用场景 | 小输入、内存受限  | 大规模卷积层（GPU优化）   |

</div>

- im2col会把输入数据展开以适合滤波器（权重）。对于**输入数据**，将应用滤波器的区域（3维方块）横向展开为**1列**。在滤波器的应用区域重叠的情况下，使用im2col展开后，展开后的元素个数会多于原方块的元素个数。因此，使用im2col的实现存在比普通的实现消耗更多内存的缺点。使用im2col展开输入数据后，之后就只需将卷积层的滤波器（权重）纵向展开为**1列**，并计算2个矩阵的乘积即可。这和全连接层的Affine 层进行的处理基本相同。
<div align="center">   <img src="data_mk\mk-2025-11-06-22-35-29.png" width="55%" style="margin: 20px 0;" /> </div>


In [ ]:
import numpy as np
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    将输入图像数据 (batch of feature maps) 转换为二维矩阵形式，以便用矩阵乘法计算卷积。

    在深度学习的卷积层（Convolution Layer）中，我们通常希望：
      - 用矩阵乘法（GEMM）来替代循环卷积操作，以加速计算。
      - 这要求我们把输入图像中每个卷积核感受野 (receptive field) 的区域展开成一个列向量。

    Parameters
    ----------
    input_data : np.ndarray
        输入数据，形状为 (N, C, H, W)
        N: 批量大小（batch size）
        C: 通道数（如 RGB = 3）
        H: 图像高度
        W: 图像宽度
    filter_h : int
        卷积核的高度
    filter_w : int
        卷积核的宽度
    stride : int
        卷积滑动的步长
    pad : int
        图像边缘填充的像素数（padding）

    Returns
    -------
    col : np.ndarray
        展开后的二维矩阵，形状为 (N*out_h*out_w, C*filter_h*filter_w)
        每一行对应卷积核在输入图像上一个位置的感受野区域。
    """
    # 1️⃣ 从输入数据中取出基本维度信息
    N, C, H, W = input_data.shape

    # 2️⃣ 计算卷积输出的特征图尺寸 (输出高度和宽度)
    # 卷积计算公式：
    # out_h = (H + 2*pad - filter_h) / stride + 1
    # out_w = (W + 2*pad - filter_w) / stride + 1
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    # 3️⃣ 在输入图像周围填充0（padding）
    # np.pad的参数 [(batch维度), (通道维度), (高维度), (宽维度)]
    # 这里在高和宽方向上各填充 pad 个像素
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

    # 4️⃣ 创建一个空矩阵，用于存放所有卷积窗口的区域
    # 每个窗口区域的形状是 (C, filter_h, filter_w)
    # 输出特征图有 out_h * out_w 个窗口，每个批次 N 个样本
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    # 5️⃣ 遍历卷积核的每个位置，将对应区域取出来放到 col 中
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            # 按步长 stride 取样：相当于卷积核在输入上滑动
            # img[..., y:y_max:stride, x:x_max:stride] 表示所有样本在当前滤波位置的值
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    # 6️⃣ 调整维度顺序 (transpose)：
    # 目标：把(N, C, filter_h, filter_w, out_h, out_w)
    # 调整为 (N*out_h*out_w, C*filter_h*filter_w)
    # 这样每一行就是一个感受野区域展开后的向量。
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)

    return col

def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """
    将 im2col 展开的二维矩阵重新还原为原始输入图像（或特征图）的形状。
    这是卷积层反向传播中常用的操作。

    在 forward（前向传播）中：
        input (N, C, H, W) -> im2col -> (N*out_h*out_w, C*filter_h*filter_w)
    在 backward（反向传播）中：
        dcol -> col2im -> (N, C, H, W)

    Parameters
    ----------
    col : np.ndarray
        通过 im2col 展开的 2D 矩阵（或它的梯度版本）
        形状通常为 (N*out_h*out_w, C*filter_h*filter_w)
    input_shape : tuple
        原始输入数据的形状，例如 (N, C, H, W)
    filter_h : int
        卷积核高度
    filter_w : int
        卷积核宽度
    stride : int
        卷积的步长
    pad : int
        填充的像素数

    Returns
    -------
    img : np.ndarray
        还原后的四维张量 (N, C, H, W)
    """
    # 1️⃣ 解包输入形状
    N, C, H, W = input_shape

    # 2️⃣ 计算卷积输出的空间尺寸
    # 与 im2col 相同的公式
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    # 3️⃣ 把 col 恢复成六维张量的形式：
    # (N, out_h, out_w, C, filter_h, filter_w)
    # 然后转置为 (N, C, filter_h, filter_w, out_h, out_w)
    # 这一步相当于还原 im2col 展开前的结构
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    # 4️⃣ 创建一个全零的图像张量（包含padding）
    # 注意尺寸：加上 2*pad 的边界，以及 stride-1 的余量，保证索引不越界
    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))

    # 5️⃣ 将每个卷积窗口（col中的值）叠加回 img 中对应的位置
    # 因为在卷积过程中，不同窗口的感受野可能会重叠，
    # 所以在还原时需要“加起来”而不是直接覆盖。
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            # += 的原因：不同卷积窗口对同一个输入像素可能有贡献（重叠部分）
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    # 6️⃣ 去掉 padding 部分，恢复原始输入大小
    return img[:, :, pad:H + pad, pad:W + pad]

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W              # 卷积核（权重），形状: (滤波器数量, 通道数, 高, 宽)
        self.b = b              # 偏置项
        self.stride = stride    # 步幅（每次移动的像素数）
        self.pad = pad          # 填充（在边缘补0）
        
        # 这些变量会在 forward 中保存中间结果，用于 backward（反向传播）
        self.x = None
        self.col = None
        self.col_W = None
        
        # 存储梯度（在反向传播时计算）
        self.dW = None
        self.db = None
    def forward(self, x):
        FN, C, FH, FW = self.W.shape   # 卷积核形状
        N, C, H, W = x.shape           # 输入数据形状 (batch数, 通道, 高, 宽)

        # 输出特征图的尺寸计算公式
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        # 将输入图像展开成二维矩阵（每个滤波区域一行）
        col = im2col(x, FH, FW, self.stride, self.pad)

        # 将卷积核展开成二维矩阵（每个滤波器一列）
        col_W = self.W.reshape(FN, -1).T

        # 矩阵乘法实现卷积（比循环快得多）
        out = np.dot(col, col_W) + self.b

        # 将输出重新变回 (N, 通道数, 高, 宽)
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        # 保存中间结果以便反向传播使用
        self.x = x
        self.col = col
        self.col_W = col_W

        return out
    def backward(self, dout):
        # ============================================================
        # 1️⃣ 取出权重形状信息
        # ============================================================
        # FN: 卷积核（滤波器）的数量，对应输出通道数
        # C : 输入的通道数（例如 RGB 图像为 3）
        # FH, FW: 卷积核的高和宽
        FN, C, FH, FW = self.W.shape


        # ============================================================
        # 2️⃣ 调整 dout 的维度以便矩阵运算
        # ============================================================
        # dout 是来自上层的梯度，形状为 (N, FN, out_h, out_w)
        # 但在 forward 时，我们将输入通过 im2col() 展开成二维矩阵，
        # 因此为了对齐矩阵乘法的维度，这里也需要将 dout 展平。
        #
        # 具体操作：
        # - 先把通道 FN 移到最后 (N, out_h, out_w, FN)
        # - 再把整个输出展平成二维矩阵 (N*out_h*out_w, FN)
        dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN)
        # 此时每一行对应一个卷积窗口的输出梯度向量（长度 = 滤波器个数 FN）。


        # ============================================================
        # 3️⃣ 计算偏置项的梯度 self.db
        # ============================================================
        # 每个卷积核都有一个偏置项 b_j，它被加到该通道所有位置上。
        # 因此偏置的梯度就是所有样本、所有空间位置的梯度求和。
        #
        # 数学表达式：
        # db_j = Σ_{n,h,w} dout[n, j, h, w]
        self.db = np.sum(dout, axis=0)  # 对所有样本求和，axis=0 表示按列求和
        # 结果形状为 (FN,)，即每个卷积核一个偏置梯度。


        # ============================================================
        # 4️⃣ 计算卷积核权重的梯度 self.dW
        # ============================================================
        # 在前向传播中，卷积计算可以表示为：
        #     out = col @ col_W + b
        # 其中：
        #     col     → 输入展开矩阵，形状 (N*out_h*out_w, C*FH*FW)
        #     col_W   → 卷积核展开矩阵，形状 (C*FH*FW, FN)
        #
        # 反向传播中，根据链式法则：
        #     dW = col^T @ dout
        # 这表示每个权重的梯度由输入与输出梯度共同决定。
        self.dW = np.dot(self.col.T, dout)
        # 上式结果形状为 (C*FH*FW, FN)，与 col_W 相同。
        #
        # 接着还原回卷积核的原始形状 (FN, C, FH, FW)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)
        # 现在 self.dW[i] 表示第 i 个卷积核的梯度矩阵。


        # ============================================================
        # 5️⃣ 计算输入展开矩阵的梯度 dcol
        # ============================================================
        # 在 forward 中：
        #     out = col @ col_W
        # 所以在 backward 时：
        #     dcol = dout @ col_W^T
        # 表示上层误差反向传播回输入展开矩阵的梯度。
        #
        # 形状对齐：
        #     dout.shape   = (N*out_h*out_w, FN)
        #     col_W.T.shape = (FN, C*FH*FW)
        #     → 结果 dcol.shape = (N*out_h*out_w, C*FH*FW)
        dcol = np.dot(dout, self.col_W.T)


        # ============================================================
        # 6️⃣ 将展开后的输入梯度还原回输入形状
        # ============================================================
        # 在 forward 时，我们使用 im2col 将输入的局部块展开为行；
        # 在 backward 时，要做逆操作，把这些梯度重新“拼”回输入图像。
        #
        # 注意：卷积窗口之间有重叠区域，col2im() 会自动将这些重叠位置的梯度相加。
        #
        # 输入参数：
        #   dcol        → 展开的输入梯度矩阵
        #   self.x.shape → 原输入形状 (N, C, H, W)
        #   FH, FW      → 卷积核大小
        #   stride, pad → 步幅和填充，与 forward 时一致
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
        # dx 即为最终传递给前一层的输入梯度。


        # ============================================================
        # 7️⃣ 返回输入梯度
        # ============================================================
        # 这样上一层就可以继续接收梯度，完成整个网络的反向传播。
        return dx

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        # ============================================================
        # 初始化池化层的超参数
        # ============================================================
        # pool_h, pool_w : 池化窗口的高和宽（例如 2x2 池化）
        # stride          : 池化的步幅（窗口每次移动的距离）
        # pad             : 填充的像素数，通常池化不使用填充（pad=0）
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        # 在前向传播中保存的中间变量（供反向传播使用）
        self.x = None        # 保存输入数据
        self.arg_max = None  # 记录每个池化区域中最大值的位置（反向传播用）


    # ============================================================
    # 前向传播（forward）
    # ============================================================
    def forward(self, x):
        # x 的形状： (N, C, H, W)
        # N: 批大小(batch size)
        # C: 通道数
        # H, W: 输入图像的高和宽
        N, C, H, W = x.shape

        # 计算输出特征图的高和宽
        # 池化层与卷积层类似，也有输出尺寸计算公式：
        # out_h = (输入高 - 池化高) / 步幅 + 1
        # out_w = (输入宽 - 池化宽) / 步幅 + 1
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 使用 im2col 将输入的所有池化窗口展开成二维矩阵
        # 每一行代表一个池化区域（例如 2x2 → 长度4的一行）
        # 展开后形状: (N*out_h*out_w*C, pool_h*pool_w)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)

        # 将每个池化窗口展开后重新整理形状
        # 把每个池化窗口的元素拉成一行方便取最大值
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # 对每一行取最大值所在的索引位置（用于反向传播）
        arg_max = np.argmax(col, axis=1)

        # 对每一行取最大值（即池化操作）
        out = np.max(col, axis=1)

        # 重新变回 (N, C, out_h, out_w) 的形状
        # 注意这里先 reshape 再 transpose，是因为 im2col 展开时的顺序
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        # 保存中间变量，反向传播时需要用
        self.x = x
        self.arg_max = arg_max

        # 返回池化结果
        return out


    # ============================================================
    # 反向传播（backward）
    # ============================================================
    def backward(self, dout):
        # dout 是上层传下来的梯度，形状为 (N, C, out_h, out_w)
        # 池化层反向传播的核心思想：
        #   前向时最大值“获胜”，反向时只有最大值位置能收到梯度，
        #   其他位置的梯度为 0。
        dout = dout.transpose(0, 2, 3, 1)
        # 现在形状为 (N, out_h, out_w, C)
        # 这样方便按输出位置对应地展开为矩阵形式。


        # ============================================================
        # 1️⃣ 计算每个池化区域的大小
        # ============================================================
        pool_size = self.pool_h * self.pool_w


        # ============================================================
        # 2️⃣ 创建与展开后的池化窗口对应的梯度矩阵
        # ============================================================
        # 目标：为每个池化区域分配梯度
        #
        # dout.size 表示总的输出元素数量 = N*out_h*out_w*C
        # 因此这里创建一个形状为 (dout.size, pool_size) 的全零矩阵，
        # 每一行对应一个池化区域（与 forward 展开的行一一对应）。
        dmax = np.zeros((dout.size, pool_size))

        # 将上层的梯度填充到最大值位置：
        #   arg_max 记录了每个池化区域最大值的索引
        #   dout.flatten() 展开成一维
        #   np.arange(self.arg_max.size) 生成行索引
        #
        # 效果：
        #  只有最大值对应的位置接收梯度，其他位置梯度为 0
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()

        # 将 dmax 的形状还原为与前向展开匹配的形状
        # 形状为 (N, out_h, out_w, C, pool_size)
        dmax = dmax.reshape(dout.shape + (pool_size,))


        # ============================================================
        # 3️⃣ 将展开后的梯度矩阵再 reshape 为 (行, 列) 形式
        # ============================================================
        # 这一步与 forward 中的 im2col 对应。
        # 把所有池化窗口的梯度重新排列成二维矩阵，准备 col2im 还原。
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)


        # ============================================================
        # 4️⃣ 将展开的梯度矩阵还原回输入形状
        # ============================================================
        # col2im 的作用：把每个窗口的梯度“放回”输入图像对应的位置。
        # 对于重叠的区域，col2im 会自动将梯度相加。
        #
        # 参数：
        #   dcol: 展开的梯度矩阵
        #   self.x.shape: 原输入形状 (N, C, H, W)
        #   self.pool_h, self.pool_w: 池化窗口大小
        #   stride, pad: 与 forward 相同
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)

        # ============================================================
        # 5️⃣ 返回输入梯度 dx
        # ============================================================
        # dx 传递给前一层，用于继续反向传播。
        return dx
